# Extract a tensor of features for debugging on the embedded device

In [4]:
import librosa as lr
import octafilt3r.filter as o3f
import os
from tensorflow.keras.models import load_model
import numpy as np
import pipeline.config as conf
import pipeline.scale as scale

In [5]:
def df2c(fp, df, class_name):

    df = df[0] # drop first dimension

    code = f"// constant features auto-generated for ai-verification. Class: [{class_name}]\n\n"

    for frame in df:
        code += "{\n"
        for db in frame:
            code += f"{db},\n"
        code += "},\n"   
    
    with open(fp, 'w') as f:
        f.write(code)



In [6]:
param_dict = conf.open_params()

class_of_interest = "sine"

base_path = "Datasets/sine_data/" + class_of_interest + "/"
stop_at = 1
i = 0
true = 0
false = 0

N4CED = load_model(param_dict["CNN_name"])

for file in os.listdir(base_path):

    if(i == stop_at):
        break

    """
    LOAD DATA
    """
    wav, fs = lr.load(base_path + file, sr=param_dict['fs'], duration=1)

    if len(wav) < fs:
        continue


    """
    EXTRACT FEATURES
    """
    feats, fcs = o3f.rolling_oct_bank(
        wav,
        fs,
        param_dict['oct_bw_ratio'],
        param_dict['order'],
        param_dict['fmax'],
        param_dict['fmin'],
        param_dict['frame_size'],
        param_dict['dec_stages'],
        param_dict['dec_ord']
    )


    """
    RESHAPE FEATURES
    """
    df = scale.feats2frames(feats, param_dict['frame_size'], fs)
    df2c("../STM32/Nu_L5_embedded_CNN_PoC_V-0-1/Core/Inc/SW/ai_verify.txt", df, class_of_interest)

    """
    SCALE FEATURES
    """
    # X, scaler = scale.data_scaler(df)
    df = df[..., np.newaxis]
    print(df.shape)


    """
    RUN MODEL
    """
    p = N4CED.predict(df)


    """
    PROCESS RESULTS
    """
    predicted = param_dict["class_map"][np.argmax(np.mean(p, axis=0))]
    print(f'Predicted class: "{predicted}" with {int(100 * (p[0][np.argmax(p, 1)][0]))} score.')
    if predicted == class_of_interest:
        true += 1
    else:
        false += 1
    i += 1

print('----- done. -----')
print(f'{true} true predictions\n{false} false predictions')
print(f'test accuracy: {int((true/i)*100)}%\nerror ratio: {int((false/i)*100)}%')

(1, 40, 28, 1)
Predicted class: "sine" with 96 score.
----- done. -----
1 true predictions
0 false predictions
test accuracy: 100%
error ratio: 0%
